In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func

In [2]:
#read in json file with groceries data
#nrows=10 for testing code
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5074155,5,True,"10 4, 2018",AGOPF2VTEOP57,B01HJHSVG6,Bob,"Love this product, very fresh, complete full f...",Exceclent product,1538611200,NaN,NaN,{'Size:': ' 2.29 oz'}
5074156,3,True,"10 3, 2018",A5X5TI4JCH9CN,B01HJHSVG6,Jacqueline Schafer,Good taste but small quantity for the expensiv...,Good taste but small quantity for the price us...,1538524800,NaN,NaN,{'Size:': ' 2.29 oz'}
5074157,4,True,"10 2, 2018",A1WMQQLC3TVAFI,B01HJHSVG6,Motionjunkie,Unlike some of the bad reviews regarding aroma...,See Comments below,1538438400,NaN,NaN,{'Size:': ' 2.29 oz'}
5074158,5,True,"09 30, 2018",ALVP6JZRTEDY6,B01HJHSVG6,Marissa,I absolutely love this vanilla bean paste. I h...,Love!!,1538265600,NaN,NaN,{'Size:': ' 2.29 oz'}


In [4]:
#read labeled csv, will be used to count words in each review to generate review length column. 
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [5]:
#count the number of words in a review and add a review length column
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))
Grocery_json.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN,270.0
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN,287.0
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN,1156.0
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN,126.0
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN,92.0


In [6]:
#create groceries_df with the relevant columns
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [7]:
#drop dupicates (drops duplicates where the whole row is identical)
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [8]:
#create df of 5 star ratings only
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5.head()

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0


In [9]:
#create df with 1 star rating only
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1.head()

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97.0
22,4639725183,1,AE937MVR5J8OL,"12 10, 2016",1481328000,Tasted old and not individually bag,104.0
27,4639725183,1,A27LF8222OIRRN,"09 10, 2016",1473465600,Not the great taste everyone was talking about :(,40.0
37,4639725183,1,A34RH95REHOQFO,"09 25, 2015",1443139200,Terrible. What a waste of money. This is not t...,1391.0
69,4639725043,1,A3ELDSSPP9RZPG,"03 31, 2017",1490918400,The tea is so fine it leaves tea particles flo...,194.0


In [10]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1659658,B005258A0K,5,AJQN9SH9FSPB1,"06 10, 2015",1433894400,Amazing breath freshener.\nDoes stick a little...,NaN
1659662,B005258A0K,5,AV37026DN7QFF,"06 6, 2015",1433548800,My daughter has started chewing gum to help ke...,NaN
1659667,B005258A0K,5,A2ID6N65Y5QYDT,"05 15, 2015",1431648000,This is the best xylitol gum on the market! L...,NaN
1659670,B005258A0K,5,A3V1S70XDFIPA,"05 7, 2015",1430956800,Nice to be able to get gum without aspartame i...,NaN


In [11]:
#another way to merge the data from 1 and 5 star ratings into one df
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
3,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
4,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1005079,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
1005080,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
1005081,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
1005082,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [12]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
# sqlite:///groc15trimmed.sqlite
engine = create_engine('sqlite:///reviewdb.db', echo=True)
sqlite_connection = engine.connect()

In [13]:
#connecting to database and inputting info in from dataframe
sqlite_table = "groceriesALL"
ALLgroceries=g1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-24 09:32:14,857 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-24 09:32:14,865 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:14,868 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-24 09:32:14,869 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:14,870 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-24 09:32:14,871 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:14,873 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-24 09:32:14,873 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:14,878 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-24 09:32:14,879 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-24 09:32:14,883 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [14]:
MLgroceries_df = g1_5[['asin', 'overall', 'reviewText']]
MLgroceries_df

,asin,overall,reviewText
0,1888861614,5,Very pleased with my purchase. Looks exactly l...
1,1888861614,5,Omg.. The S was inexpensive and EXACTLY what w...
2,1888861614,5,This was exactly what we were looking for to m...
3,4639725183,5,No adverse comment.
4,4639725183,5,Best black tea in US.\n\nHighly recommend.\nI ...
...,...,...,...
1005079,B01HJ7QPKU,1,Followed instructions on the package to prepar...
1005080,B01HJ4R980,1,For my grandson. Arrived broken into crumbs.
1005081,B01HJ4R980,1,These are delicious cookies - just don't order...
1005082,B01HJ4R980,1,Can't say he product is good as is practically...


In [15]:
#connecting to database and inputting info in from dataframe
sqlite_table_2 = "groceriesMLData"
MLgroceries=MLgroceries_df.to_sql(sqlite_table_2, sqlite_connection, if_exists='replace')

2021-06-24 09:32:39,778 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-24 09:32:39,778 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:39,780 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-24 09:32:39,781 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:39,783 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-24 09:32:39,784 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:39,786 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesMLData")
2021-06-24 09:32:39,787 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:32:39,788 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-24 09:32:39,789 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesMLData',)
2021-06-24 09:32:39,791 INFO sqlalchemy.engine.Engine PRAGMA

In [16]:
#select and show data in groceries ALL table
#engine.execute('SELECT * FROM groceriesALL').fetchall()

In [17]:
#engine.execute('SELECT * FROM groceriesMLData').fetchall()